In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import scipy
from scipy.stats import norm
import tensorflow_probability as tfp
import seaborn as sns
import pandas as pd
import corner 
tfd=tfp.distributions

In [18]:
@tf.function
def A(x,z,p1,p2):
    numerator = (x[p2]-x[p1])*(x[p1]*z[p2]-x[p2]*z[p1])
    denominator = (x[p2]-x[p1])**2 + (z[p2]-z[p1])**2
    return (numerator/denominator)

@tf.function
def B(x,z,p1,p2):
    return ((z[p1]-z[p2])/(x[p2]-x[p1]))
@tf.function
def theta(x,z, p):
    if x[p] != 0 :
        if(np.arctan(z[p]/x[p]))<0:
            return(tf.atan(z[p]/x[p])+scipy.pi)
        else:
            return(tf.atan(z[p]/x[p]))
    if x[p] == 0 and z[p]!=0 :
        return(scipy.pi/2)
@tf.function
def r(x,z,p):
    return(np.sqrt(x[p]**2+z[p]**2))
@tf.function
def Z(x,z,p1,p2):
    
    if x[p1] == z[p1]==0 or x[p2] == z[p2]==0:
        return(0)
    
    if x[p1] == x[p2]:
        return((x[p1]*tf.math.log(r(x,z,p2)/r(x,z,p1))))
    
    theta1 = theta(x,z, p1)
    theta2 = theta(x,z, p2)
    
    r1 = r(x,z,p1)
    r2 = r(x,z,p2)
    
    #this is only needed when observation location across z
#     if z[p1]*z[p2]<0:
#         if x[p1]*z[p2]==x[p2]*z[p1]:
#             return(0)
#         if x[p1]*z[p2] < x[p2]*z[p1] and z[p2]>=0:
#             theta1 += 2*scipy.pi
#         if x[p1]*z[p2] > x[p2]*z[p1] and z[p1]>=0:
#             theta2 += 2*scipy.pi
    
    _A = A(x,z,p1,p2)
    _B = B(x,z,p1,p2)
    
    Z_result = _A*((theta1-theta2)+_B*tf.math.log(r1/r2))
    return(Z_result)

@tf.function
def g(data,loc=(0,0)):
    
    G = 6.67 * 10**(-11) # gravitational constant  m^3 kg ^-1 s^-2
    rho = 1000        # density difference   kg/m^3
    
    _data = data - loc #Calculate any point refer to the origin

    _x = _data[:,0]
    _z = _data[:,1]

    Z_sum = 0
    
    for i in range(_data.shape[0]-1):
        Z_sum += Z(_x,_z,i,i+1)

    Z_sum += Z(_x,_z,-1,0)

    g = 2*G*rho * Z_sum
    
    return(g)

@tf.function
def grav(base,ps):
    
    #base_points:2xN tensor contains x location
    #ps: 1xN vector, elevation variate
    
    for i in range(N):
        base[i+4][1] = ps[i]
        
    x_obv = np.linspace(-70., 70., 11)
    y_obv = np.zeros(np.shape(x_obv))
    obv = np.vstack((x_obv,y_obv)).T
    
    gravity = []
    for i in obv[1:-1]:
        gravity.append(-g(base,i))
        
    return gravity

@tf.function
def joint_log_prob(D,points_copy,ps):
    """
    D: is the observation data
    ps: is the variable point positions (N elements vector)
    """
    # define random variables prior
    mvn_prior = tfd.Uniform(
            low = low_,
            high = high_)
    # define likelihood
    
    Gm_ = grav(points_copy,ps)
    Gm = tf.convert_to_tensor(Gm_,dtype)
    
    mvn_likelihood = tfd.MultivariateNormalFullCovariance(
            loc = Gm,
            covariance_matrix= cov)
    
    # return the posterior probability
    return(tf.reduce_sum(mvn_prior.log_prob(ps))
          +mvn_likelihood.log_prob(D))

In [19]:
x = [0.,2.,3.,4.]
z = [-1.,-2.,-3.,-4.]
x = tf.convert_to_tensor(x,dtype = tf.float32)
z = tf.convert_to_tensor(z,dtype = tf.float32)

In [23]:
if x[1] == x[1] or x[1]:
    print('e')

e


In [51]:
@tf.function
def theta(x,z, p):
    s = 0.
    if tf.math.logical_and(a,b):
        s+=x[p]
    return(s)
a = theta(x,z,0)

In [52]:
a

<tf.Tensor: id=744, shape=(), dtype=float32, numpy=0.0>